## Telephone production: a descriptive model

A possible descriptive model of the telephone production problem is as follows:
- Decision variables:
    - Number of desk phones produced (DeskProduction)
    - Number of cellular phones produced (CellProduction)
- Objective: Maximize profit
- Constraints:
    1. The DeskProduction should be greater than or equal to 100.
    2. The CellProduction should be greater than or equal to 100.
    3. The assembly time for DeskProduction plus the assembly time for CellProduction should not exceed 400 hours.
    4. The painting time for DeskProduction plus the painting time for CellProduction should not exceed 490 hours.


## Use IBM Decision Optimization CPLEX Modeling for Python

Let's use the DOcplex Python library to write the mathematical model in Python.

### Step 1: Download the library

First install *docplex* if needed.

In [ ]:
import sys
try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install docplex
    else:
        !pip install --user docplex

### Step 2: Set up the prescriptive engine

* Subscribe to our private cloud offer or Decision Optimization on Cloud solve service [here](https://developer.ibm.com/docloud) if you do not want to use a local solver.
* Get the service URL and your personal API key and enter your credentials here if accurate:

In [ ]:
url = "https://api-oaas.docloud.ibmcloud.com/job_manager/rest/v1/"
key = "api_44f5ba41-b02b-479a-bec2-3ccb0ee6927a"

### Step 3: Set up the prescriptive model
#### Create the model

All objects of the model belong to one model instance.

In [ ]:
# first import the Model class from docplex.mp
from docplex.mp.model import Model

# create one model instance, with a name
m = Model(name='telephone_production')

#### Define the decision variables
- The continuous variable `desk` represents the production of desk telephones.
- The continuous variable `cell` represents the production of cell phones.

In [ ]:
# by default, all variables in Docplex have a lower bound of 0 and infinite upper bound
desk = m.continuous_var(name='desk')
cell = m.continuous_var(name='cell')

#### Set up the constraints
- Desk and cel phone must both be greater than 100
- Assembly time is limited
- Painting time is limited.

In [ ]:
# write constraints
# constraint #1: desk production is greater than 100
m.add_constraint(desk >= 100)

# constraint #2: cell production is greater than 100
m.add_constraint(cell >= 100)

# constraint #3: assembly time limit
ct_assembly = m.add_constraint( 0.2 * desk + 0.4 * cell <= 400)

# constraint #4: paiting time limit
ct_painting = m.add_constraint( 0.5 * desk + 0.4 * cell <= 490)

#### Express the objective

We want to maximize the expected revenue.

In [ ]:
m.maximize(12 * desk + 20 * cell)

In [ ]:
m.print_information()

### Solve with the Decision Optimization solve service

If url and key are None, the Modeling layer will look for a local runtime, otherwise will use the credentials.

Look at the documentation for a good understanding of the various solving/generation modes.

If you're using a Community Edition of CPLEX runtimes, depending on the size of the problem, the solve stage may fail and will need a paying subscription or product installation.

In any case, `Model.solve()` returns a solution object in Python, containing the optimal values of decision variables, if the solve succeeds, or else it returns `None`.

In [ ]:
s = m.solve(url=url, key=key)
m.print_solution()

In this case, CPLEX has found an optimal solution at (300, 850). You can check that this point is indeed an extreme point of the feasible region.

### Multiple Optimal Solutions

It is possible that an LP has multiple optimal solutions. 
At least one optimal solution will be at a vertex.
By default, the CPLEX® Optimizer reports the first optimal solution found. 


#### Infeasible models in DOcplex

Calling `solve()` on an infeasible model returns None. Let's experiment this with DOcplex. First, we take a copy of our model and an extra infeasible constraint which states that desk telephone production must be greater than 1100

In [ ]:
# create a new model, copy of m
im = m.copy()
# get the 'desk' variable of the new model from its name
idesk = im.get_var_by_name('desk')
# add a new (infeasible) constraint
im.add_constraint(idesk >= 1100);
# solve the new proble, we expect a result of None as the model is now infeasible
ims = im.solve(url=url, key=key)
if ims is None:
    print('- model is infeasible')

### Correcting infeasible models

To correct an infeasible model, you must use your knowledge of the real-world situation you are modeling.
If you know that the model is realizable, you can usually manually construct an example of a feasible solution and use it to determine where your model or data is incorrect. For example, the telephone production manager may input the previous month's production figures as a solution to the model and discover that they violate the erroneously entered bounds of 1100.

DOcplex can help perform infeasibility analysis, which can get very complicated in large models. In this analysis, DOcplex may suggest relaxing one or more constraints. 


#### Implement the soft constraint model using DOcplex

First and extra variable for overtime, with an upper bound of 100. This suffices to express the hard limit on overtime.

In [ ]:
overtime = m.continuous_var(name='overtime', ub=40)

Modify the assembly time constraint by changing its right-hand side by adding overtime. 

*Note*: this operation modifies the model by performing a _side-effect_ on the constraint object. DOcplex allows dynamic edition of model elements.

In [ ]:
ct_assembly.rhs = 400 + overtime

Last, modify the objective expression to add the penalization term.
Note that we use the Python decrement operator.

In [ ]:
m.maximize(12*desk + 20 * cell - 2 * overtime)

And solve again using DOcplex:

In [ ]:
s2 = m.solve(url=url, key=key)
m.print_solution()

### The Dual simple algorithm

#### The dual of a LP

The concept of duality is important in linear programming.  Every LP problem has an associated LP problem known as its _dual_. The dual of this associated problem is the original LP problem (known as the primal problem). If the primal problem is a minimization problem, then the dual problem is a maximization problem and vice versa.

#### A primal-dual pair

<p>
<ul>
<img src = "https://ibmdecisionoptimization.github.io/tutorials/jupyter/training/42.png?raw=true" >
</ul> 


*Primal (P)*      
-------------------- 

 $max\ z=\sum_{i} c_{i}x_{i}$  
 
*Dual (D)*
-------------------------------
 $min\ w= \sum_{j}b_{j}y_{j}$ 
 

- Each constraint in the primal has an associated dual variable, yi.
- Any feasible solution to D is an upper bound to P, and any feasible solution to P is a lower bound to D.
- In LP, the optimal objective values of D and P are equivalent, and occurs where these bounds meet.
- The dual can help solve difficult primal problems by providing a bound that in the best case equals the optimal solution to the primal problem.

#### Dual prices

In any solution to the dual, the values of the dual variables are known as the dual prices, also called shadow prices.

For each constraint in the primal problem, its associated dual price indicates how much the dual objective will change with a unit change in the right hand side of the constraint.

The dual price of a non-binding constraint is zero. That is, changing the right hand side of the constraint will not affect the objective value.

The dual price of a binding constraint can help you make decisions regarding the constraint.

For example, the dual price of a binding resource constraint can be used to determine whether more of the resource should be purchased or not.

#### The dual simplex algorithm

The simplex algorithm works by finding a feasible solution and moving progressively toward optimality. 

The dual simplex algorithm implicitly uses the dual to try and find an optimal solution to the primal as early as it can, and regardless of whether the solution is feasible or not. 

It then moves from one vertex to another, gradually decreasing the infeasibility while maintaining optimality, until an optimal feasible solution to the primal problem is found. 

In CPLEX, the Dual-simplex Optimizer is the first choice for most LP problems.


#### Getting reduced cost values with DOcplex

DOcplex lets you access reduced costs of variable, after a successful solve. Let's experiment with the two decision variables of our problem:

In [ ]:
print('* desk variable has reduced cost: {0}'.format(desk.reduced_cost))
print('* cell variable has reduced cost: {0}'.format(cell.reduced_cost))

### Default optimality criteria for CPLEX optimizer

Because CPLEX Optimizer operates on finite precision computers, it uses an optimality tolerance to test the reduced costs. 

The default optimality tolerance is –1e-6, with optimality criteria for the simplest form of an LP then being:

$$
c — y^{t}A> –10^{-6}
$$

You can adjust this optimality tolerance, for example if the algorithm takes very long to converge and has already achieved a solution sufficiently close to optimality.


### Reduced Costs and multiple optimal solutions

In the earlier example you saw how one can visualize multiple optimal solutions for an LP with two variables. 
For larger LPs, the reduced costs can be used to determine whether multiple optimal solutions exist. Multiple optimal solutions exist when one or more non-basic variables with a zero reduced cost exist in an optimal solution (that is, variable values that can change without affecting the objective value). 
In order to determine whether multiple optimal solutions exist, you can examine the values of the reduced costs with DOcplex. 

#### Accessing slack values with DOcplex

As an example, let's examine the slack values of some constraints in our problem, after we revert the change to soft constrants

In [ ]:
# revert soft constraints
ct_assembly.rhs = 440
s3 = m.solve(url=url, key=key)

# now get slack value for assembly constraint: expected value is 40
print('* slack value for assembly time constraint is: {0}'.format(ct_assembly.slack_value))
# get slack value for painting time constraint, expected value is 0.
print('* slack value for painting time constraint is: {0}'.format(ct_painting.slack_value))

#### Setting a LP algorithm with DOcplex

Users can change the algorithm by editing the `lpmethod` parameter of the model.
We won't go into details here, it suffices to know this parameter accepts an integer from 0 to 6, where 0 denotes automatic choice of the algorithm, 1 is for primal simplex, 2 is for dual simplex, and 4 is for barrier...

For example, choosing the barrier algorithm is done by setting value 4 to this parameter. We access the `parameters` property of the model and from there, assign the `lpmethod` parameter

In [ ]:
m.parameters.lpmethod = 4
m.solve(url=url, key=key, log_output=True)